### 五年人口迁徙图

In [1]:
import re
import pandas as pd
import requests as rq
from iclientpy import viz

使用pandas读取数据，但是列错乱，修正后生成新的dataframe

In [2]:
sdf = pd.read_html(rq.get('http://www.stats.gov.cn/tjsj/pcsj/rkpc/6rp/html/B0708.htm', headers={'User-agent': 'Mozilla/5.0'}).content)[0]
df=sdf.iloc[8:39,[0]+list(range(3,34))]
df.columns=['现住地']+sdf.iloc[5,1:32].tolist()

从源数据获取各地方迁出总人口的dataframe，然后提取迁出人口最多的五个省份

In [3]:
tdf=sdf.iloc[7:8,3:34]
tdf.columns=sdf.iloc[5,1:32].tolist()
tdf.loc[7]=pd.to_numeric(tdf.loc[7])
ttdf=tdf.T
ttdf.columns=['value']
sttdf=ttdf.sort_values(by='value',ascending=False)
top5_name=sttdf.iloc[0:5].index.tolist()

构建geolines所需要的数据格式

In [4]:
def get_data(dataframe):
    alldata=[]
    for index,series in dataframe.iteritems():
        if index !=  "现住地":
            single_data=[]
            for ind,value in series.iteritems():
                if  index != re.sub('\s','',dataframe['现住地'][ind]):
                   single_data.append([{'name': index},{'name': re.sub('\s','',dataframe['现住地'][ind]),'value':value}]) 
            alldata.append(single_data)
    return alldata

In [5]:
data=get_data(df)

生成迁徙图

In [6]:
chart=viz.geolines(data=data,address_key='name',value_key='value',names=df.columns[1:].tolist(),max_symbolsize=15,min_symbolsize=5,symbol='arrow',selected_mode='multiple',selected_legend=top5_name)

展示迁徙图

In [7]:
chart

MapView(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': '…

迁徙出去人口的等级符号专题图

In [8]:
ttdf['地区']=ttdf.index.tolist()
out_chart=viz.ranksymboltheme(data=ttdf,address_key='地区',value_key='value',is_over_lay=False,codomain=(0,400000),fill_opacity=0.5)
out_chart

MapView(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': '…

In [9]:
out_chart.interact()

计算各个地区迁入人口的数据

In [10]:
idf=sdf.iloc[8:39,3:34]
idf=idf.apply(pd.to_numeric, errors='ignore')
idf=idf.fillna(0)
itdf=idf.T.apply(sum)
itdf.columns=sdf.iloc[5,1:32].tolist()
idf=pd.DataFrame([sdf.iloc[5,1:32].tolist(),itdf.data.tolist()]).T
idf.columns=['地区','value']

各个地区迁入人口的等级符号专题图

In [11]:
in_chart=viz.ranksymboltheme(data=idf,address_key='地区',value_key='value',is_over_lay=False,codomain=(0,400000))
in_chart

MapView(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': '…

In [12]:
in_chart.interact()